# Proyek Analisis Data: **Bike Sharing Dataset**
- **Nama:** Muhammad Husain Fadhlillah
- **Email:** mc006d5y2343@student.devacademy.id
- **ID Dicoding:** MC006D5Y2343

## **Menentukan Pertanyaan Bisnis**

- Pertanyaan 1: Bagaimana pola penggunaan sepeda (bike sharing) berdasarkan faktor waktu (jam, hari, bulan, musim) dan faktor cuaca?
- Pertanyaan 2: Apa perbedaan karakteristik antara pengguna casual dan registered, dan bagaimana ini dapat mempengaruhi strategi bisnis?

## **Import Semua Packages/Library yang Digunakan**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import calendar

# Untuk visualisasi yang lebih baik
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette('viridis')

## **Data Wrangling**

### **Gathering Data**

In [ ]:
# Load dataset
hour_data = pd.read_csv('hour.csv')
day_data = pd.read_csv('day.csv')

# Cek struktur data
print("Hour Data Shape:", hour_data.shape)
print("Day Data Shape:", day_data.shape)

print("\nHour Data Preview:")
hour_data.head()

**Insight:**

1. **Struktur Dataset**:
   - Dataset hour memiliki 17.379 baris dan 17 kolom, menandakan pencatatan per jam selama periode tertentu
   - Dataset day memiliki 731 baris dan 16 kolom, menunjukkan data harian selama kurang lebih 2 tahun

2. **Informasi Waktu**:
   - Data yang ditampilkan adalah dari tanggal 1 Januari 2011 pada jam 0-4 pagi
   - Hari tersebut adalah weekday (nilai 6, kemungkinan Sabtu) dan bukan hari libur (holiday = 0)
   - Data tercatat saat musim 1 (winter/musim dingin)

3. **Pola Penggunaan Sepeda**:
   - Jam 1 pagi (hr = 0) menunjukkan total 16 pengguna (cnt)
   - Jam 2 pagi (hr = 1) menunjukkan lonjakan signifikan hingga 40 pengguna
   - Penggunaan menurun setelahnya: 32, 13, dan hanya 1 pengguna di jam 5 pagi
   - Ini mendukung kesimpulan bahwa pola penggunaan sepeda dipengaruhi oleh waktu

4. **Karakteristik Pengguna**:
   - Pengguna terdaftar (registered) mendominasi dibanding pengguna kasual (casual)
   - Pada jam 1 pagi: 13 registered vs 3 casual
   - Pada jam 2 pagi: 32 registered vs 8 casual
   - Pada jam 3 pagi: 27 registered vs 5 casual
   - Pada jam 4 pagi: 10 registered vs 3 casual
   - Pada jam 5 pagi: 1 registered vs 0 casual
   - Ini memperkuat kesimpulan bahwa pengguna registered mendominasi

5. **Faktor Cuaca**:
   - Suhu (temp) berkisar antara 0.22-0.24 (nilai normalisasi)
   - Suhu yang dirasakan (atemp) sekitar 0.27-0.29
   - Kelembaban (hum) berkisar 0.75-0.81
   - Kecepatan angin (windspeed) konsisten 0.0
   - Weathersit konsisten bernilai 1 (clear/partly cloudy)
   - Kondisi cuaca yang baik mendukung penggunaan sepeda

**Hubungan dengan Pertanyaan Bisnis:**
- Pertanyaan 1: Data menunjukkan variasi penggunaan berdasarkan jam, meskipun pada malam/dini hari. Kondisi cuaca yang baik (weathersit=1) memungkinkan penggunaan sepeda bahkan di malam hari.
- Pertanyaan 2: Proporsi pengguna registered jauh lebih tinggi daripada casual, mengonfirmasi kesimpulan bahwa pengguna registered mendominasi.

### **Assessing Data**

In [ ]:
# Cek missing values
print("Missing values in hour data:")
print(hour_data.isnull().sum())

print("\nMissing values in day data:")
print(day_data.isnull().sum())

# Cek duplikasi data
print("\nDuplicates in hour data:", hour_data.duplicated().sum())
print("Duplicates in day data:", day_data.duplicated().sum())

# Cek nilai unik pada kolom kategorikal
categorical_cols = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
for col in categorical_cols:
    print(f"\nUnique values in {col}: {hour_data[col].unique()}")
    print(f"Count: {hour_data[col].nunique()}")

# Cek range tanggal
print("\nDate range in hour data:")
print("Min date:", hour_data['dteday'].min())
print("Max date:", hour_data['dteday'].max())

**Insight:**

1. **Kelengkapan dan Kualitas Data**:
   - Tidak ada missing values pada kedua dataset (hour dan day)
   - Tidak ditemukan data duplikat pada kedua dataset
   - Hal ini menunjukkan kualitas data yang baik dan siap untuk dianalisis lebih lanjut

2. **Rentang Waktu Dataset**:
   - Dataset mencakup periode 2 tahun penuh: 1 Januari 2011 hingga 31 Desember 2012
   - Rentang waktu yang cukup panjang memungkinkan analisis tren musiman dan tahunan yang komprehensif

3. **Variabel Kategoris dan Dimensi Waktu**:
   - **Season**: 4 nilai unik (1-4) menunjukkan empat musim dalam setahun
   - **Year**: 2 nilai unik (0-1) menunjukkan data selama 2 tahun
   - **Month**: 12 nilai unik (1-12) menunjukkan semua bulan dalam setahun
   - **Hour**: 24 nilai unik (0-23) menunjukkan data lengkap untuk setiap jam dalam sehari
   - Variabel waktu ini sangat relevan dengan Pertanyaan Bisnis 1 tentang pola penggunaan berdasarkan waktu

4. **Variabel Status Hari**:
   - **Holiday**: 2 nilai unik (0-1) menunjukkan hari libur dan non-libur
   - **Weekday**: 7 nilai unik (0-6) menunjukkan semua hari dalam seminggu
   - **Workingday**: 2 nilai unik (0-1) membedakan hari kerja dan akhir pekan/libur
   - Variabel ini mendukung analisis perbedaan perilaku pengguna casual vs registered pada hari kerja vs akhir pekan (Pertanyaan Bisnis 2)

5. **Kondisi Cuaca**:
   - **Weathersit**: 4 nilai unik (1-4) menunjukkan variasi kondisi cuaca
   - Variabel ini krusial untuk menganalisis pengaruh cuaca terhadap penggunaan sepeda (Pertanyaan Bisnis 1)

**Hubungan dengan Pertanyaan Bisnis**:
   - **Pertanyaan 1**: Dataset memiliki semua variabel waktu (jam, hari, bulan, musim) dan cuaca yang diperlukan untuk menganalisis pola penggunaan sepeda. Kelengkapan data (tidak ada missing values) memperkuat validitas kesimpulan bahwa pola penggunaan dipengaruhi oleh waktu dan cuaca.
   
   - **Pertanyaan 2**: Ketersediaan variabel seperti workingday, weekday, dan holiday memungkinkan analisis mendalam tentang perbedaan perilaku antara pengguna casual dan registered di hari kerja versus akhir pekan. Kesimpulan bahwa "pengguna casual lebih aktif di akhir pekan" dapat divalidasi dengan membandingkan proporsi pengguna pada nilai workingday=0 (akhir pekan/libur) dan workingday=1 (hari kerja).

### **Cleaning Data**

In [ ]:
# 1. Ubah tipe data kolom datetime
import os
hour_data['dteday'] = pd.to_datetime(hour_data['dteday'])
day_data['dteday'] = pd.to_datetime(day_data['dteday'])

# 2. Mapping kolom kategorikal
season_map = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}
hour_data['season_name'] = hour_data['season'].map(season_map)
day_data['season_name'] = day_data['season'].map(season_map)

year_map = {0: '2011', 1: '2012'}
hour_data['year'] = hour_data['yr'].map(year_map)
day_data['year'] = day_data['yr'].map(year_map)

month_map = {i: calendar.month_name[i] for i in range(1, 13)}
hour_data['month_name'] = hour_data['mnth'].map(month_map)
day_data['month_name'] = day_data['mnth'].map(month_map)

weekday_map = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday'}
hour_data['weekday_name'] = hour_data['weekday'].map(weekday_map)
day_data['weekday_name'] = day_data['weekday'].map(weekday_map)

weather_map = {
    1: 'Clear/Few clouds',
    2: 'Mist/Cloudy',
    3: 'Light Snow/Rain',
    4: 'Heavy Rain/Snow/Fog'
}
hour_data['weather_condition'] = hour_data['weathersit'].map(weather_map)
day_data['weather_condition'] = day_data['weathersit'].map(weather_map)

# 3. Denormalisasi suhu, kelembaban, dan kecepatan angin
hour_data['temp_actual'] = hour_data['temp'] * (39 - (-8)) + (-8)
day_data['temp_actual'] = day_data['temp'] * (39 - (-8)) + (-8)

hour_data['atemp_actual'] = hour_data['atemp'] * (50 - (-16)) + (-16)
day_data['atemp_actual'] = day_data['atemp'] * (50 - (-16)) + (-16)

hour_data['hum_actual'] = hour_data['hum'] * 100
day_data['hum_actual'] = day_data['hum'] * 100

hour_data['windspeed_actual'] = hour_data['windspeed'] * 67
day_data['windspeed_actual'] = day_data['windspeed'] * 67

# 4. Tambahkan kolom total pengguna dan proporsi
hour_data['total_users'] = hour_data['casual'] + hour_data['registered']
day_data['total_users'] = day_data['casual'] + day_data['registered']

hour_data['casual_proportion'] = hour_data['casual'] / hour_data['total_users'] * 100
hour_data['registered_proportion'] = hour_data['registered'] / hour_data['total_users'] * 100

day_data['casual_proportion'] = day_data['casual'] / day_data['total_users'] * 100
day_data['registered_proportion'] = day_data['registered'] / day_data['total_users'] * 100

os.makedirs('dashboard', exist_ok=True)

# Simpan data bersih
hour_data.to_csv('dashboard/hour_data_clean.csv', index=False)
day_data.to_csv('dashboard/day_data_clean.csv', index=False)

**Insight:**

1. **Transformasi Tipe Data Datetime**:
   - Data tanggal diubah ke format datetime standar menggunakan pd.to_datetime()
   - Ini memudahkan analisis deret waktu untuk mengidentifikasi pola penggunaan sepeda berdasarkan waktu

2. **Peningkatan Interpretabilitas Data Kategori**:
   - Kolom kategori numerik dikonversi menjadi label deskriptif:
     - Musim: 1-4 → Spring, Summer, Fall, Winter
     - Tahun: 0-1 → 2011, 2012
     - Bulan: 1-12 → January-December
     - Hari: 0-6 → Sunday-Saturday
     - Cuaca: 1-4 → Clear/Few clouds, Mist/Cloudy, Light Snow/Rain, Heavy Rain/Snow/Fog
   - Transformasi ini memudahkan pemahaman pola penggunaan sepeda berdasarkan faktor waktu dan cuaca

3. **Denormalisasi Data Cuaca**:
   - Suhu: dikonversi dari range 0-1 ke nilai aktual (-8°C hingga 39°C)
   - Suhu yang dirasakan: dikonversi ke range -16°C hingga 50°C
   - Kelembaban: dikonversi ke persentase (0-100%)
   - Kecepatan angin: dikonversi ke nilai aktual (0-67 km/h)
   - Denormalisasi ini memungkinkan interpretasi yang lebih intuitif tentang pengaruh cuaca terhadap pola penggunaan

4. **Perhitungan Proporsi Pengguna**:
   - Ditambahkan kolom proporsi pengguna casual dan registered (dalam persentase)
   - Ini memungkinkan analisis perbandingan distribusi kedua jenis pengguna tanpa terpengaruh oleh perbedaan jumlah absolut

5. **Validasi Total Pengguna**:
   - Penambahan kolom total_users sebagai jumlah dari pengguna casual dan registered
   - Ini menyediakan cross-check untuk memastikan konsistensi data

**Hubungan dengan Pertanyaan Bisnis**:
   - **Pertanyaan 1**: Transformasi data waktu (musim, bulan, hari, jam) dan cuaca memberikan dasar yang kuat untuk menganalisis pola penggunaan sepeda berdasarkan kedua faktor tersebut. Denormalisasi variabel cuaca memungkinkan analisis yang lebih mendalam tentang kondisi spesifik yang mempengaruhi penggunaan sepeda.
   
   - **Pertanyaan 2**: Perhitungan proporsi pengguna casual vs registered memberi perspektif yang lebih jelas tentang perbedaan karakteristik kedua kelompok ini. Kombinasi data proporsi dengan label hari dalam seminggu (weekday_name) akan sangat membantu dalam memvalidasi kesimpulan bahwa pengguna casual lebih aktif di akhir pekan.

## **Exploratory Data Analysis (EDA)**

### Analisis Pola Waktu

In [ ]:
# Pola penggunaan sepeda berdasarkan jam
hourly_pattern = hour_data.groupby('hr')['cnt'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(x='hr', y='cnt', data=hourly_pattern, marker='o', linewidth=2)
plt.title('Rata-rata Penyewaan Sepeda berdasarkan Jam', fontsize=15)
plt.xlabel('Jam', fontsize=12)
plt.ylabel('Rata-rata Jumlah Penyewaan', fontsize=12)
plt.xticks(range(0, 24))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Insight:**

1. **Pola Bi-modal yang Jelas**:
   - Terdapat dua puncak utama penggunaan: pagi (jam 8) dan sore/malam (jam 17-18)
   - Puncak tertinggi terjadi pada jam 17-18 (sekitar 450 penyewaan)
   - Puncak kedua terjadi pada jam 8 pagi (sekitar 350 penyewaan)
   - Pola ini menunjukkan korelasi kuat dengan jam berangkat dan pulang kerja/sekolah

2. **Periode Penggunaan Terendah**:
   - Penggunaan paling rendah terjadi pada dini hari (jam 3-4) dengan kurang dari 10 penyewaan
   - Penurunan signifikan terlihat setelah jam 19 hingga mencapai titik terendah pada jam 3-4
   - Jam 23 masih menunjukkan angka penyewaan sekitar 90 unit

3. **Pola Penggunaan Siang Hari**:
   - Setelah puncak pagi (jam 8), terjadi penurunan hingga jam 10 (sekitar 175 penyewaan)
   - Dari jam 10 hingga 15, penggunaan relatif stabil di kisaran 240-260 penyewaan
   - Mulai jam 15, terjadi kenaikan tajam hingga mencapai puncak di jam 17-18

4. **Transisi Antar Periode**:
   - Peningkatan tajam terjadi antara jam 5-8 pagi (dari 20 menjadi 350 penyewaan)
   - Periode 10-15 relatif stabil sebagai "plateau" di tengah hari
   - Peningkatan signifikan terlihat lagi pada jam 15-17
   - Penurunan gradual dari jam 18-23 menunjukkan pengurangan aktivitas malam hari

### Analisis Pengaruh Cuaca

In [ ]:
# Pengaruh kondisi cuaca
weather_impact = hour_data.groupby('weather_condition')['cnt'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.barplot(x='weather_condition', y='cnt', data=weather_impact, palette='viridis')
plt.title('Rata-rata Penyewaan Sepeda berdasarkan Kondisi Cuaca', fontsize=15)
plt.xlabel('Kondisi Cuaca', fontsize=12)
plt.ylabel('Rata-rata Jumlah Penyewaan', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Insight:**

1. **Pengaruh Signifikan Kondisi Cuaca**:
   - Terdapat variasi yang jelas dalam rata-rata penyewaan sepeda di berbagai kondisi cuaca
   - Kondisi "Clear/Few clouds" memiliki rata-rata penyewaan tertinggi (sekitar 205 unit)
   - Kondisi "Heavy Rain/Snow/Fog" menunjukkan penyewaan terendah (sekitar 75 unit)
   - Hal ini mengkonfirmasi bahwa cuaca menjadi faktor penentu dalam keputusan pengguna

2. **Urutan Preferensi Kondisi Cuaca**:
   - Dari paling disukai ke paling dihindari: Clear/Few clouds > Mist/Cloudy > Light Snow/Rain > Heavy Rain/Snow/Fog
   - Perbedaan antara kondisi terbaik dan terburuk sangat signifikan (hampir 3 kali lipat)
   - Pengguna masih cukup aktif pada kondisi Mist/Cloudy (sekitar 175 unit)

3. **Toleransi terhadap Kondisi Cuaca**:
   - Pengguna cenderung toleran terhadap kondisi berawan (Mist/Cloudy) dengan perbedaan tidak terlalu jauh dari kondisi cerah
   - Hujan/salju ringan (Light Snow/Rain) menyebabkan penurunan moderat (sekitar 110 unit)
   - Kondisi cuaca ekstrem (Heavy Rain/Snow/Fog) memberikan efek penghambat paling kuat

4. **Implikasi Operasional**:
   - Perbedaan penyewaan antara kondisi terbaik dan terburuk hampir 130 unit (63% penurunan)
   - Cuaca menjadi faktor risiko yang signifikan dalam prediksi permintaan sepeda

## **Visualization & Explanatory Analysis**

## Pertanyaan 1: Pola Penggunaan Sepeda

In [ ]:
# Heatmap jam vs hari
hour_weekday_heatmap = hour_data.pivot_table(index='hr', columns='weekday_name', values='cnt', aggfunc='mean')
hour_weekday_heatmap = hour_weekday_heatmap.reindex(columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

plt.figure(figsize=(16, 10))
sns.heatmap(hour_weekday_heatmap, cmap='viridis', annot=False, fmt='.0f', cbar_kws={'label': 'Rata-rata Penyewaan'})
plt.title('Heatmap Penyewaan Sepeda: Jam vs Hari', fontsize=16)
plt.xlabel('Hari', fontsize=12)
plt.ylabel('Jam', fontsize=12)
plt.tight_layout()
plt.show()

**Insight:**

1. **Pola Jam Sibuk pada Hari Kerja**
   - Terdapat 2 peak hours yang jelas pada hari Senin-Jumat: pukul 8 pagi dan sekitar pukul 17-18 (jam pulang kerja)
   - Pola ini menunjukkan bahwa sepeda banyak digunakan untuk commuting (perjalanan ke/dari tempat kerja)

2. **Perbedaan Pola Weekday vs Weekend**
   - Hari kerja (Senin-Jumat) memiliki pola penggunaan yang berbentuk "twin peaks" (pagi dan sore)
   - Akhir pekan (Sabtu-Minggu) memiliki distribusi yang lebih merata sepanjang siang hingga sore hari

3. **Penggunaan pada Malam Hari**
   - Terjadi penurunan signifikan penggunaan sepeda mulai pukul 22.00 hingga 05.00 pagi di semua hari
   - Area berwarna ungu gelap menunjukkan penggunaan paling rendah pada dini hari

4. **Peak Penggunaan pada Akhir Pekan**
   - Pukul 11.00-18.00 pada Sabtu-Minggu memiliki tingkat penyewaan yang tinggi (warna hijau-kuning)
   - Menunjukkan penggunaan lebih untuk rekreasi dibandingkan commuting

5. **Hari dengan Penggunaan Tertinggi**
   - Hari kerja pada jam sibuk (terutama sore hari) dan hari Sabtu siang menunjukkan puncak penyewaan tertinggi (warna kuning/hijau terang)

6. **Gradasi Penggunaan Berdasarkan Waktu**
   - Penggunaan meningkat secara bertahap dari pagi hari, mencapai puncak di sore hari, dan menurun di malam hari

### Pertanyaan 2: Perbedaan Pengguna Casual dan Registered

In [ ]:
# Perbandingan pengguna casual vs registered
total_users = day_data[['casual', 'registered']].sum()
labels = ['Casual', 'Registered']

plt.figure(figsize=(10, 8))
plt.pie(total_users, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#ff9999', '#66b3ff'])
plt.title('Proporsi Pengguna Casual vs Registered', fontsize=15)
plt.axis('equal')
plt.tight_layout()
plt.show()

**Insight:**

1. **Dominasi Pengguna Registered**
   - Pengguna registered mendominasi dengan proporsi 81,2% dari total penyewaan sepeda
   - Hanya 18,8% penyewaan dilakukan oleh pengguna casual
   - Perbandingan ini menunjukkan rasio sekitar 4:1 antara pengguna registered dan casual

2. **Pangsa Pasar yang Tidak Seimbang**
   - Terdapat kesenjangan yang sangat besar antara kedua jenis pengguna
   - Pengguna registered menghasilkan lebih dari 4/5 dari total transaksi penyewaan

3. **Indikasi Loyalitas Pelanggan**
   - Tingginya persentase pengguna registered menunjukkan keberhasilan dalam membangun basis pelanggan tetap
   - Mengindikasikan bahwa mayoritas pengguna adalah pelanggan berulang yang memiliki komitmen jangka panjang

4. **Potensi Pertumbuhan dari Pengguna Casual**
   - Segmen pengguna casual (18,8%) mewakili peluang konversi untuk meningkatkan basis pengguna registered
   - Meskipun kecil, segmen ini penting sebagai pintu masuk pengguna baru ke dalam sistem

5. **Stabilitas Pendapatan**
   - Dominasi pengguna registered menunjukkan basis pendapatan yang lebih stabil dan dapat diprediksi
   - Layanan memiliki arus pendapatan yang lebih terjamin dari basis pelanggan tetap

## **Analisis Lanjutan (Opsional)**

In [ ]:
# Clustering berdasarkan pola penggunaan
X = hour_data[['hr', 'cnt']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3)
hour_data['cluster'] = kmeans.fit_predict(X_scaled)

plt.figure(figsize=(12, 6))
sns.scatterplot(x='hr', y='cnt', hue='cluster', data=hour_data, palette='viridis')
plt.title('Clustering Pola Penggunaan Sepeda', fontsize=15)
plt.xlabel('Jam', fontsize=12)
plt.ylabel('Jumlah Penyewaan', fontsize=12)
plt.tight_layout()
plt.show()

**Insight:**

1. **Identifikasi 3 Cluster Penggunaan yang Berbeda**
   - Cluster 0 (warna ungu): Penggunaan menengah, dominan pada malam hari (jam 11-23)
   - Cluster 1 (warna hijau): Penggunaan tinggi, dominan pada jam sibuk (terutama jam 7-9 dan 16-19)
   - Cluster 2 (warna kuning): Penggunaan rendah, dominan pada dini hari (jam 0-5)

2. **Pola Jam Sibuk yang Jelas**
   - Konsentrasi tertinggi penyewaan (500-980 unit) terjadi pada jam 7-9 dan 16-19
   - Cluster 1 (hijau) mencapai puncak hampir 1000 penyewaan pada jam 17-18
   - Pola ini menunjukkan hubungan kuat dengan jam berangkat dan pulang kerja

3. **Penggunaan di Waktu Malam**
   - Penurunan bertahap dari jam 19-23 dengan range 250-600 penyewaan
   - Cluster 0 (ungu) mendominasi periode malam dengan jumlah penyewaan menengah
   - Relatif stabil dibandingkan fluktuasi pada jam kerja

4. **Penggunaan Sangat Rendah di Dini Hari**
   - Cluster 2 (kuning) menunjukkan aktivitas paling rendah pada jam 0-5 (<300 penyewaan)
   - Memperlihatkan periode "idle" yang signifikan dalam sistem penyewaan

5. **Transisi Antar Periode**
   - Terjadi transisi cluster yang jelas: kuning → hijau (pagi), hijau → ungu (malam)
   - Jam 6-7 dan 19-20 menunjukkan periode transisi dimana terjadi pergeseran pola penggunaan

6. **Variasi dalam Cluster**
   - Cluster 1 (hijau) menunjukkan variasi vertikal paling tinggi, menandakan fluktuasi besar dalam volume penyewaan pada jam sibuk
   - Faktor eksternal (cuaca, hari kerja vs akhir pekan) kemungkinan mempengaruhi variasi ini

## **Conclusion**

- Kesimpulan Pertanyaan 1: Pola penggunaan sepeda dipengaruhi oleh waktu dan cuaca.
- Kesimpulan Pertanyaan 2: Pengguna registered mendominasi, sedangkan casual lebih aktif di akhir pekan.